In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# Turning the csv into a DataFrame
houses = pd.read_csv('data/kc_house_data.csv')

In [19]:
#houses.head()

In [18]:
#houses.info()

In [3]:
def city(address):
    address_list = address.split(',')
    if len(address_list) == 6:
        city = address_list[3].strip()
    elif len(address_list) == 5:
        city = address_list[2].strip()
    else:
        city = address_list[1].strip()
    return city

def state(address):
    address_list = address.split(',')
    if len(address_list) == 6:
        statezip = address_list[4].split()
    elif len(address_list) == 5:
        statezip = address_list[3].split()
    else:
        statezip = address_list[2].split()
        
    if len(statezip) > 2:
        state = statezip[0] + ' ' + statezip[1]
    else:
        state = statezip[0]
    return state

def zipcode(address):
    address_list = address.split(',')
    zipcode = address_list[-2].split()[-1]
    return zipcode

In [4]:
houses['city'] = houses['address'].map(city)
houses['zipcode'] = houses['address'].map(zipcode)
houses['state'] = houses['address'].map(state)

In [24]:
#Changing date to datetime and then creating a separate column for year
houses['date'] = pd.to_datetime(houses['date'], infer_datetime_format=True)
# feature engineering 
# making a 'yr_sold' column using the year from 'date'
houses['yr_sold'] = pd.to_datetime(houses['date'], infer_datetime_format=True).dt.year

In [26]:
"""
Looking at the correlation between all variables to see which ones have the strongest corr
with variable 'price'.
"""
houses.corr()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,lat,long,yr_sold
id,1.000000,-0.034124,-0.006322,-0.012023,-0.027880,-0.119109,0.032075,-0.023183,-0.014601,-0.007807,-0.041615,0.023125,-0.029140,-0.000681,0.000473,-0.009106
price,-0.034124,1.000000,0.289241,0.480347,0.608491,0.085742,0.180542,0.538634,0.244991,0.264150,0.313407,0.095945,0.084802,0.063621,-0.022501,0.073957
bedrooms,-0.006322,0.289241,1.000000,0.589335,0.637918,0.003304,0.147605,0.547186,0.238536,0.319452,0.183444,0.146215,0.014283,0.108762,-0.106691,-0.027088
bathrooms,-0.012023,0.480347,0.589335,1.000000,0.772659,0.035896,0.404390,0.674917,0.260828,0.457014,0.327553,0.443602,0.040647,-0.005241,0.017410,-0.041768
sqft_living,-0.027880,0.608491,0.637918,0.772659,1.000000,0.119574,0.304215,0.883983,0.338410,0.511732,0.396030,0.291649,0.038510,0.102178,-0.087664,-0.029025
sqft_lot,-0.119109,0.085742,0.003304,0.035896,0.119574,1.000000,-0.032094,0.129236,0.004118,0.087173,0.155252,0.001756,0.010048,0.030021,-0.034309,-0.004700
floors,0.032075,0.180542,0.147605,0.404390,0.304215,-0.032094,1.000000,0.448268,-0.248149,0.132643,0.125177,0.544632,-0.025444,-0.218563,0.233787,-0.017370
sqft_above,-0.023183,0.538634,0.547186,0.674917,0.883983,0.129236,0.448268,1.000000,-0.066852,0.560544,0.312113,0.387425,0.010605,0.092252,-0.082741,-0.023044
sqft_basement,-0.014601,0.244991,0.238536,0.260828,0.338410,0.004118,-0.248149,-0.066852,1.000000,0.026330,0.210494,-0.230316,0.053648,0.059777,-0.045202,-0.009374
sqft_garage,-0.007807,0.264150,0.319452,0.457014,0.511732,0.087173,0.132643,0.560544,0.026330,1.000000,0.216349,0.447550,-0.098599,0.092342,-0.096859,-0.012718


# Observations about R^2

- small blurb about highest R^2 and potential multicollinearity between variables, such as sqft_living and sqft_above

# Data Cleaning

- small blurb explaining why we cleaned and which ones we picked

In [27]:
clean_houses = houses.loc[houses['state'] == 'Washington']
clean_houses.drop_duplicates(subset=['id'], inplace=True)

<ipython-input-27-d8f60f4d9dbb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_houses.drop_duplicates(subset=['id'], inplace=True)


In [28]:
clean_houses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29244 entries, 0 to 30154
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             29244 non-null  int64         
 1   date           29244 non-null  datetime64[ns]
 2   price          29244 non-null  float64       
 3   bedrooms       29244 non-null  int64         
 4   bathrooms      29244 non-null  float64       
 5   sqft_living    29244 non-null  int64         
 6   sqft_lot       29244 non-null  int64         
 7   floors         29244 non-null  float64       
 8   waterfront     29244 non-null  object        
 9   greenbelt      29244 non-null  object        
 10  nuisance       29244 non-null  object        
 11  view           29244 non-null  object        
 12  condition      29244 non-null  object        
 13  grade          29244 non-null  object        
 14  heat_source    29214 non-null  object        
 15  sewer_system   2923

In [29]:
def Rsquaredcomp(dataframe,target_str,comp_list):
    Rsquared = []
    for i in comp_list:
        y = dataframe[target_str]
        X = dataframe[i]
        simple_model = sm.OLS(endog=y, exog=sm.add_constant(X))
        simple_model_results = simple_model.fit()
        r2 = simple_model_results.rsquared
        Rsquared.append(r2)
    
    df = pd.DataFrame(np.array(Rsquared),index=comp_list,columns=["Rsquared"])
    return df

In [30]:
comp = ['floors','bedrooms','bathrooms','floors','sqft_living', 'sqft_above','sqft_basement','sqft_garage','sqft_patio', 'sqft_lot',
        'yr_built', 'yr_renovated','lat','long']
Rsquaredcomp(clean_houses,'price',comp)

,Rsquared
floors,0.039654
bedrooms,0.084849
bathrooms,0.238156
floors,0.039654
sqft_living,0.380203
sqft_above,0.298243
sqft_basement,0.060771
sqft_garage,0.071796
sqft_patio,0.100619
sqft_lot,0.007393


In [31]:
# Dropping columns with low individual R^2
clean_houses = clean_houses.drop(columns=['date','sqft_above','sqft_basement','sqft_garage','sqft_patio',
                                              'sqft_lot','waterfront','greenbelt','nuisance','condition',
                                              'heat_source','sewer_system','state','yr_built',
                                              'yr_renovated','address','lat','long','floors'])

In [32]:
clean_houses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29244 entries, 0 to 30154
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           29244 non-null  int64  
 1   price        29244 non-null  float64
 2   bedrooms     29244 non-null  int64  
 3   bathrooms    29244 non-null  float64
 4   sqft_living  29244 non-null  int64  
 5   view         29244 non-null  object 
 6   grade        29244 non-null  object 
 7   city         29244 non-null  object 
 8   zipcode      29244 non-null  object 
 9   yr_sold      29244 non-null  int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 2.5+ MB


In [23]:
#clean_houses.loc[clean_houses['id'] == 1233100736]
#checking to make sure that the duplicate is gone

In [22]:
#Rerunning the corr to see if anything has changed, whoops have to dummy many variables
#clean_houses.corr()

In [33]:
cat_df = pd.get_dummies(clean_houses,columns=['view','grade','city','zipcode'])

In [34]:
cat_df

,id,price,bedrooms,bathrooms,sqft_living,yr_sold,view_AVERAGE,view_EXCELLENT,view_FAIR,view_GOOD,...,zipcode_98338,zipcode_98354,zipcode_98372,zipcode_98387,zipcode_98422,zipcode_98663,zipcode_99202,zipcode_99203,zipcode_99223,zipcode_99403
0,7399300360,675000.0,4,1.0,1180,2022,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8910500230,920000.0,5,2.5,2770,2021,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1180000275,311000.0,6,2.0,2880,2021,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1604601802,775000.0,3,3.0,2160,2021,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8562780790,592500.0,2,2.0,1120,2021,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30150,7834800180,1555000.0,5,2.0,1910,2021,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30151,194000695,1313000.0,3,2.0,2020,2021,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
30152,7960100080,800000.0,3,2.0,1620,2022,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30153,2781280080,775000.0,3,2.5,2570,2022,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
"""
Talking about next steps, bar graph of yr sold as supporting visual.
Subset 2021 2022, avg sale price
Then subset into the zipcodes

"""